In [1]:
import os
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import keras
import tensorflow as tf
import cv2

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing import image
import pickle

In [2]:
labels_path = "./data/labels2"
images_data_path = "./data/images2"

In [3]:
# def create_df():
#     data = {
#         'image_id' : [],
#         'labels':[]
#     }
#     for filename in os.listdir(images_data_path):
#         data['image_id'].append(filename)
        
#         label_file = os.path.splitext(filename)[0] + '.txt'
#         label_path = os.path.join(labels_path, label_file)

#         with open(label_path, 'r') as f:
#             label_data = f.readlines()
        
#         label = []
#         for line in label_data:
#             parts = line.split()
#             x_min, y_min, x_max, y_max = map(float, parts[2:])
#             label.append([x_min, y_min, x_max, y_max])
#         data['labels'].append(label)
#     df = pd.DataFrame(data)
    
#     return df

In [ ]:
# df = create_df()
# df.head()

In [5]:
# def plotting_image_with_box(index):

#     test_image = cv2.imread(images_data_path + "/" + df["image_id"][index])
#     points_list = df["labels"][index]

#     fig, ax = plt.subplots()

#     for points in points_list:
#         xtl, ytl, xbr, ybr = points[0], points[1], points[2], points[3]

#         rect = patches.Rectangle((xtl, ytl), xbr - xtl, ybr - ytl, linewidth=2, edgecolor='g', facecolor='none')
#         ax.add_patch(rect)

#     ax.imshow(test_image)

In [ ]:
# plotting_image_with_box(3)

In [3]:
def load_data(images_path, labels_path):
    images = []
    labels = []
    # i = 0
    for filename in os.listdir(images_path):
        img_path = os.path.join(images_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        # i = i + 1
        # if (i == 50): break
        original_height, original_width = img.shape[:2]
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        
        x_scale = 224 / original_width
        y_scale = 224 / original_height

        label_file = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(labels_path, label_file)
        
        if not os.path.exists(label_path):
            continue
        
        with open(label_path, 'r') as f:
            label_data = f.readlines()
        
        label = []
        for line in label_data:
            parts = line.split()
            x_min, y_min, x_max, y_max = map(float, parts[2:])
            x_min_resized = x_min * x_scale
            y_min_resized = y_min * y_scale
            x_max_resized = x_max * x_scale
            y_max_resized = y_max * y_scale
            label.append([x_min_resized, y_min_resized, x_max_resized, y_max_resized])
        
        images.append(img)
        labels.append(label)
    
    X = np.array(images, dtype=np.float32)
    
    # Define max_faces based on your use case or dynamically calculate it
    max_faces = max(len(label) for label in labels) if labels else 0
    Y = np.zeros((len(labels), max_faces * 4), dtype=np.float32)
    
    for i, label in enumerate(labels):
        flattened_labels = np.array(label).flatten()
        Y[i, :len(flattened_labels)] = flattened_labels
    
    X_tensor = tf.convert_to_tensor(X)
    Y_tensor = tf.convert_to_tensor(Y)
    
    return X_tensor, Y_tensor

In [4]:
X, y = load_data(images_data_path, labels_path)
print(y[0])

tf.Tensor(
[ 72.8       51.449886 103.95011   94.5      133.34988   49.7
 165.2       92.04989    0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.       

In [5]:
print(X.shape)
print(y.shape)

(16733, 224, 224, 3)
(16733, 612)


In [6]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
def feature_extraction(input_shape=(224, 224, 3)):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    return Model(inputs, x)

In [7]:
from tensorflow.keras.layers import GlobalAveragePooling2D
def detection_head(features, max_faces=10):
    x = Dense(256, activation='relu')(features)
    
    # Bounding box çıktıları
    bounding_boxes = Dense(max_faces * 4, activation='sigmoid')(x)  # (None, 40)
    
    # Confidence score çıktıları
    confidences = Dense(max_faces*4, activation='sigmoid')(x)  # (None, 10)
    
    
    # Bounding box ve confidence score'ları birleştir
    outputs =[bounding_boxes, confidences]  # (None, 80)
    
    return outputs

In [8]:
feature_extractor = feature_extraction()

input_image = Input(shape=(224, 224, 3))
features = Dense(1024, activation='relu')(input_image)  # Örnek bir feature extractor
outputs = detection_head(features, max_faces=10)

model = Model(inputs=input_image, outputs=outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 224, 224,  │      4,096 │ input_layer_1[0]… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 224, 224,  │    262,400 │ dense[0][0]       │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 224, 224,  │     10,280 │ dense_1[0][0]     │
│                     │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 224, 224,  │     10,280 │ dense_1[0][0]     │
│                     │ 40)               │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 287,056 (1.10 MB)

 Trainable params: 287,056 (1.10 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X.numpy(), y.numpy(), test_size=0.17,random_state = 42)

In [10]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)

In [11]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [12]:
training_process = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=100, shuffle=True)

Epoch 1/10


ValueError: Dimensions must be equal, but are 612 and 40 for '{{node compile_loss/mean_squared_error/sub}} = Sub[T=DT_FLOAT](data_1, functional_1_1/dense_2_1/Sigmoid)' with input shapes: [?,612], [?,224,224,40].

In [ ]:
with open('04Model.pkl', 'wb') as f:
    pickle.dump(model, f)